#### Loading Pre-Split Train/Test Data, 500 Pre-Sampled Train/Test Data and Model

In [1]:
import joblib
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2  
from sklearn.metrics import confusion_matrix

# Loading the pre-split data for reference 
X_train, X_test, y_train, y_test = joblib.load(r"Train_Test_Splits.pkl")

# Loading the trained model 
model = joblib.load(r"MainModel_1L_MLP.pkl")

# Loading the sampled unseen data (500 samples of test set) for MLP with one layer
X_sampled,y_sampled=joblib.load(r"Unseen_500_Sampled_data.pkl")

# Loading the sampled seen data (500 samples of train set) for MLP with one layer 
train_X_sampled,train_y_sampled=joblib.load(r"Seen_500__Sampled_data.pkl")


# Loading the sampled seen data (100 samples of train set) for MLP with one layer 
train10_X_sampled,train10_y_sampled=joblib.load(r"Seen_100__Sampled_data.pkl")


#### Evaluating Model Performance for Reference with Pre-Split Data

In [3]:
from sklearn.metrics import confusion_matrix

# Evaluate the model on the test data
y_pred = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred, axis=1)  # Convert predictions to class labels

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  # 'macro' averages for multi-class classification
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Print evaluation metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

print("--------------")


# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# False positives for each class (sum of columns except the diagonal)
false_positives = cm.sum(axis=0) - np.diagonal(cm)

# Print false positives for each class
print("False Alarms for each class:", false_positives)
print("False Alarms for all classes:", false_positives.sum())


Accuracy: 0.9739
Precision: 0.9737
Recall: 0.9736
F1 Score: 0.9736
--------------
False Alarms for each class: [14 17 46 46 30 47 21 50 38 57]
False Alarms for all classes: 366


### CSA

In [5]:

img_shape = (28, 28) # For MNIST images we use 28x28 grayscale.

# --- 1. Define the Adversarial Loss Function ---
def adversarial_loss(model, x, y):
    x_adv = x 
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure values are in [0,1]
    
    # Get predictions and compute loss
    y_pred = model(x_adv, training=False)
    y_true = tf.convert_to_tensor(y, dtype=tf.int64)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss

# --- 2. Cuckoo Search Algorithm for Finding Adversarial Perturbations ---
class CuckooSearch:
    def __init__(self, n_nests, n_iterations, dim, lower_bound, upper_bound, model, X, y, p_a=0.1):
        self.n_nests = n_nests
        self.n_iterations = n_iterations
        self.dim = dim
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.model = model
        self.X = X
        self.y = y
        self.p_a = p_a  # Discovery probability
        
        self.nests = np.random.uniform(low=0, high=1, size=(self.n_nests, self.dim)) 
        self.fitness = np.array([self.fitness_function(self.nests[i], X, y)
                                 for i in range(self.n_nests)])
        
        self.best_nest = self.nests[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)
    
    def fitness_function(self, perturbation, X, y):
        perturbed_X = X + perturbation
        perturbed_X = np.clip(perturbed_X, self.lower_bound, self.upper_bound)
        loss = adversarial_loss(self.model, perturbed_X, y)  
        perturbation_magnitude = np.linalg.norm(perturbation) 
        return - loss * perturbation_magnitude
        
    def generate_new_solution(self, current_solution, step_size=0.001):
        step = np.random.standard_normal(self.dim)**(1/1.5)    
        new_solution = current_solution + step
        new_solution = np.clip(new_solution, self.lower_bound, self.upper_bound)
        return new_solution
    
    def optimize(self):
        for iteration in range(self.n_iterations):
            # Perturb each nest
            for i in range(self.n_nests):
                new_solution = self.generate_new_solution(self.nests[i], step_size=0.001)
                new_fitness = self.fitness_function(new_solution, self.X, self.y)
                if new_fitness < self.fitness[i]:
                    self.nests[i] = new_solution
                    self.fitness[i] = new_fitness
                    if new_fitness < self.best_fitness:
                        self.best_fitness = new_fitness
                        self.best_nest = new_solution
            
            # Randomly replace some nests with new random solutions
            for i in range(self.n_nests):
                if np.random.rand() < self.p_a:
                    worst_indices = np.argsort(self.fitness)[-int(self.n_nests * self.p_a):]
                    # Replace only the worst nests
                    for idx in worst_indices:
                        self.nests[idx] = np.random.uniform(self.lower_bound, self.upper_bound, self.dim)
                        self.fitness[idx] = self.fitness_function(self.nests[idx], self.X, self.y)                   
            #print(f"Iteration {iteration+1}/{self.n_iterations}, Best Fitness: {self.best_fitness}")    
    
    def get_best_solution(self):
        return self.best_nest

# --- 3. Utility Functions for Saving Images and Perturbations ---
def create_directories(base_dir):
    adversarial_dir = os.path.join(base_dir, "Adversarial")
    os.makedirs(adversarial_dir, exist_ok=True)
    return adversarial_dir

def save_image(image, file_path, img_shape=None):
    if img_shape is not None and image.size == np.prod(img_shape):
        image = image.reshape(img_shape)
    # Scale [0,1] to [0,255] and convert to uint8
    image_to_save = (image * 255).astype(np.uint8)
    if image_to_save.ndim == 2:
        cv2.imwrite(file_path, image_to_save)
    elif image_to_save.ndim == 3:
        cv2.imwrite(file_path, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


# --- 4. Main Process: Generate, Save, and Record Adversarial Examples ---
results = []
base_dir = "CSA_images"
adversarial_dir = create_directories(base_dir)
itemCounter=0

for i in range(len(train10_X_sampled)):  
    original_vector = train10_X_sampled[i]  # e.g., shape (784,) for MNIST
    train_y_sample = train10_y_sampled[i]
    train_X_sample = original_vector.reshape(1, -1)
    
    cuckoo_search = CuckooSearch(n_nests=58, n_iterations=100, dim=original_vector.shape[0],
                                 lower_bound=0, upper_bound=0.5, model=model, X=train_X_sample, y=[train_y_sample])
    cuckoo_search.optimize()
    best_solution = cuckoo_search.get_best_solution()
    
    # Generate the adversarial example.
    X_adv = original_vector + best_solution
    X_adv = np.clip(X_adv, 0, 1)
    
    X_adv_batch = X_adv.reshape(1, -1)
    y_pred_adv = np.argmax(model.predict(X_adv_batch), axis=1)
    
    if y_pred_adv[0] != train_y_sample:
  
        perturbation_magnitude = np.linalg.norm(best_solution)
        adversarial_image_filename = f"{i}adv_orig{train_y_sample}_adv{int(y_pred_adv[0])}_mag{perturbation_magnitude}_{i}.png"
        adversarial_image_path = os.path.join(adversarial_dir, adversarial_image_filename)
        
        save_image(X_adv, adversarial_image_path, img_shape=img_shape)
        itemCounter+=1
        
        results.append({
            'Image_Index': i,
            'Original':train_y_sample,
            'Adversarial_Digit': int(y_pred_adv[0]),
            'Perturbation_Magnitude': perturbation_magnitude,
            'Adversarial_Image_Path': adversarial_image_path
        })
        print(f"Image {i} PERTURBED. itemCounter: {itemCounter } perturbation_magnitude: {perturbation_magnitude}")
    else:
        print(f"Image {i} did not result in adversarial misclassification---------------------------------.")

# Save results to an Excel file
df = pd.DataFrame(results)
df.to_excel(r"CSA_images/CSA.xlsx")

# Display the results
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Adversarial Samples Results", dataframe=df)

C:\Users\dyari\AppData\Local\Temp\ipykernel_4520\1169649464.py:42: RuntimeWarning: invalid value encountered in power
  step = np.random.standard_normal(self.dim)**(1/1.5)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 0 PERTURBED. itemCounter: 1 perturbation_magnitude: 16.369600638825798
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Image 1 PERTURBED. itemCounter: 2 perturbation_magnitude: 16.464107544950675
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 2 PERTURBED. itemCounter: 3 perturbation_magnitude: 16.10950816997061
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Image 3 PERTURBED. itemCounter: 4 perturbation_magnitude: 16.426169320469263
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Image 4 PERTURBED. itemCounter: 5 perturbation_magnitude: 16.852596250637866
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 5 PERTURBED. itemCounter: 6 perturbation_magnitude: 16.371521366501867
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image 6 PERTURBED. itemCounter: 7 perturbation_magnitude: 16.31949603985054
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 7 PERTURBED. itemCounter: 8 perturbation_magnitude: 16.51078904987329
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Image 8 PERTURBED. itemCounter: 9 perturbatio

### CSA with Zero

In [10]:

img_shape = (28, 28) # For MNIST images we use 28x28 grayscale.

# --- 1. Define the Adversarial Loss Function ---
def adversarial_loss(model, x, y):
    x_adv = x
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure values are in [0,1]
    
    # Get predictions and compute loss
    y_pred = model(x_adv, training=False)
    y_true = tf.convert_to_tensor(y, dtype=tf.int64)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss

# --- 2. Cuckoo Search Algorithm for Finding Adversarial Perturbations ---
class CuckooSearch:
    def __init__(self, n_nests, n_iterations, dim, lower_bound, upper_bound, model, X, y, p_a=0.1):
        self.n_nests = n_nests
        self.n_iterations = n_iterations
        self.dim = dim
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.model = model
        self.X = X
        self.y = y
        self.p_a = p_a  # Discovery probability
        
        # Initialize nests (solutions) with no perturbation (Zero)
        self.nests = np.zeros((self.n_nests, self.dim))

        self.fitness = np.array([self.fitness_function(self.nests[i], X, y)
                                 for i in range(self.n_nests)])
        
        self.best_nest = self.nests[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)
    
    def fitness_function(self, perturbation, X, y):
        perturbed_X = X + perturbation
        perturbed_X = np.clip(perturbed_X, self.lower_bound, self.upper_bound)
        loss = adversarial_loss(self.model, perturbed_X, y)  
        perturbation_magnitude = np.linalg.norm(perturbation)
        return - loss * perturbation_magnitude
       
    
    def generate_new_solution(self, current_solution, step_size=0.001):
        step = np.random.standard_normal(self.dim)**(1/1.5)         
        new_solution = current_solution + step
        new_solution = np.clip(new_solution, self.lower_bound, self.upper_bound)
        return new_solution
    
    def optimize(self):
        for iteration in range(self.n_iterations):
            # Perturb each nest
            for i in range(self.n_nests):
                new_solution = self.generate_new_solution(self.nests[i], step_size=0.001)
                new_fitness = self.fitness_function(new_solution, self.X, self.y)
                if new_fitness < self.fitness[i]:
                    self.nests[i] = new_solution
                    self.fitness[i] = new_fitness
                    if new_fitness < self.best_fitness:
                        self.best_fitness = new_fitness
                        self.best_nest = new_solution
            
            # Randomly replace some nests with new random solutions
            for i in range(self.n_nests):
                if np.random.rand() < self.p_a:
                    worst_indices = np.argsort(self.fitness)[-int(self.n_nests * self.p_a):]
                    # Replace only the worst nests
                    for idx in worst_indices:
                        self.nests[idx] = np.random.uniform(self.lower_bound, self.upper_bound, self.dim)
                        self.fitness[idx] = self.fitness_function(self.nests[idx], self.X, self.y)                  
            #print(f"Iteration {iteration+1}/{self.n_iterations}, Best Fitness: {self.best_fitness}")
        
    def get_best_solution(self):
        return self.best_nest

# --- 3. Utility Functions for Saving Images and Perturbations ---
def create_directories(base_dir):
    adversarial_dir = os.path.join(base_dir, "Adversarial")
    os.makedirs(adversarial_dir, exist_ok=True)  
    return adversarial_dir

def save_image(image, file_path, img_shape=None):
    if img_shape is not None and image.size == np.prod(img_shape):
        image = image.reshape(img_shape)
    # Scale [0,1] to [0,255] and convert to uint8
    image_to_save = (image * 255).astype(np.uint8)
    if image_to_save.ndim == 2:
        cv2.imwrite(file_path, image_to_save)
    elif image_to_save.ndim == 3:
        cv2.imwrite(file_path, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


# --- 4. Main Process: Generate, Save, and Record Adversarial Examples ---
results = []
base_dir = "CSAzero_images"
adversarial_dir = create_directories(base_dir)
itemCounter=0

for i in range(len(train10_X_sampled)):  
    original_vector = train10_X_sampled[i]  # e.g., shape (784,) for MNIST
    train_y_sample = train10_y_sampled[i] 
    train_X_sample = original_vector.reshape(1, -1)
    
    cuckoo_search = CuckooSearch(n_nests=58, n_iterations=100, dim=original_vector.shape[0],
                                 lower_bound=0, upper_bound=0.5, model=model, X=train_X_sample, y=[train_y_sample])
    cuckoo_search.optimize()
    best_solution = cuckoo_search.get_best_solution()
    
    # Generate the adversarial example.
    X_adv = original_vector + best_solution
    X_adv = np.clip(X_adv, 0, 1)
    
    X_adv_batch = X_adv.reshape(1, -1)
    y_pred_adv = np.argmax(model.predict(X_adv_batch), axis=1)
    
    if y_pred_adv[0] != train_y_sample:
  
        perturbation_magnitude = np.linalg.norm(best_solution)
        
        adversarial_image_filename = f"{i}adv_orig{train_y_sample}_adv{int(y_pred_adv[0])}_mag{perturbation_magnitude}_{i}.png"
        adversarial_image_path = os.path.join(adversarial_dir, adversarial_image_filename)
        
        save_image(X_adv, adversarial_image_path, img_shape=img_shape)
        itemCounter+=1
        
        results.append({
            'Image_Index': i,
            'Original':train_y_sample,
            'Adversarial_Digit': int(y_pred_adv[0]),
            'Perturbation_Magnitude': perturbation_magnitude,
            'Adversarial_Image_Path': adversarial_image_path
        })
        print(f"Image {i} PERTURBED. itemCounter: {itemCounter } perturbation_magnitude: {perturbation_magnitude}")
    else:
        print(f"Image {i} did not result in adversarial misclassification---------------------------------.")

# Save results to an Excel file
df = pd.DataFrame(results)
df.to_excel(r"CSAzero_images/CSA_Zero.xlsx")

# Display the results
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Adversarial Samples Results", dataframe=df)

C:\Users\dyari\AppData\Local\Temp\ipykernel_4520\4221346812.py:45: RuntimeWarning: invalid value encountered in power
  step = np.random.standard_normal(self.dim)**(1/1.5)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image 0 PERTURBED. itemCounter: 1 perturbation_magnitude: 8.010012376229305
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image 1 PERTURBED. itemCounter: 2 perturbation_magnitude: 8.022624951364646
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Image 2 PERTURBED. itemCounter: 3 perturbation_magnitude: 8.126807786134739
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image 3 PERTURBED. itemCounter: 4 perturbation_magnitude: 7.996768119166571
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image 4 PERTURBED. itemCounter: 5 perturbation_magnitude: 8.346842152636665
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Image 5 PERTURBED. itemCounter: 6 perturbation_magnitude: 8.171621441879008
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Image 6 PERTURBED. itemCounter: 7 perturbation_magnitude: 8.191574319231322
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Image 7 PERTURBED. itemCounter: 8 perturbation_magnitude: 8.222523241128126
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Image 8 PERTURBED. itemCounter: 9 perturbation_mag

### Pure CSA with Zero plus standard_cauchy

In [4]:

img_shape = (28, 28) # For MNIST images we use 28x28 grayscale.

# --- 1. Define the Adversarial Loss Function ---
def adversarial_loss(model, x, y):
    x_adv = x
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure values are in [0,1]
    
    # Get predictions and compute loss
    y_pred = model(x_adv, training=False)
    y_true = tf.convert_to_tensor(y, dtype=tf.int64)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss

# --- 2. Cuckoo Search Algorithm for Finding Adversarial Perturbations ---
class CuckooSearch:
    def __init__(self, n_nests, n_iterations, dim, lower_bound, upper_bound, model, X, y, p_a=0.1):
        self.n_nests = n_nests
        self.n_iterations = n_iterations
        self.dim = dim
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.model = model
        self.X = X
        self.y = y
        self.p_a = p_a  # Discovery probability
        
        # Initialize nests (solutions) with no perturbation (Zero)
        self.nests = np.zeros((self.n_nests, self.dim))
    
        self.fitness = np.array([self.fitness_function(self.nests[i], X, y)
                                 for i in range(self.n_nests)])
        
        self.best_nest = self.nests[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)
    
    def fitness_function(self, perturbation, X, y):
        perturbed_X = X + perturbation
        perturbed_X = np.clip(perturbed_X, self.lower_bound, self.upper_bound)
        loss = adversarial_loss(self.model, perturbed_X, y)  
        perturbation_magnitude = np.linalg.norm(perturbation)
        return - loss * perturbation_magnitude
       
    
    def generate_new_solution(self, current_solution, step_size=0.001):
        step = np.random.standard_cauchy(self.dim) * step_size
        new_solution = current_solution + step
        new_solution = np.clip(new_solution, self.lower_bound, self.upper_bound)
        return new_solution
    
    def optimize(self):
        for iteration in range(self.n_iterations):
            # Perturb each nest
            for i in range(self.n_nests):
                new_solution = self.generate_new_solution(self.nests[i], step_size=0.001)
                new_fitness = self.fitness_function(new_solution, self.X, self.y)
                if new_fitness < self.fitness[i]:
                    self.nests[i] = new_solution
                    self.fitness[i] = new_fitness
                    if new_fitness < self.best_fitness:
                        self.best_fitness = new_fitness
                        self.best_nest = new_solution
            
            # Randomly replace some nests with new random solutions
            for i in range(self.n_nests):
                if np.random.rand() < self.p_a:
                    worst_indices = np.argsort(self.fitness)[-int(self.n_nests * self.p_a):]
                    # Replace only the worst nests
                    for idx in worst_indices:
                        self.nests[idx] = np.random.uniform(self.lower_bound, self.upper_bound, self.dim)
                        self.fitness[idx] = self.fitness_function(self.nests[idx], self.X, self.y)        
            #print(f"Iteration {iteration+1}/{self.n_iterations}, Best Fitness: {self.best_fitness}")
        
    
    def get_best_solution(self):
        return self.best_nest

# --- 3. Utility Functions for Saving Images and Perturbations ---
def create_directories(base_dir):
    adversarial_dir = os.path.join(base_dir, "Adversarial")
    os.makedirs(adversarial_dir, exist_ok=True)
    
    return adversarial_dir

def save_image(image, file_path, img_shape=None):
    if img_shape is not None and image.size == np.prod(img_shape):
        image = image.reshape(img_shape)
    # Scale [0,1] to [0,255] and convert to uint8
    image_to_save = (image * 255).astype(np.uint8)
    if image_to_save.ndim == 2:
        cv2.imwrite(file_path, image_to_save)
    elif image_to_save.ndim == 3:
        cv2.imwrite(file_path, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


# --- 4. Main Process: Generate, Save, and Record Adversarial Examples ---
results = []
base_dir = "CSAzeroCau_images"
adversarial_dir = create_directories(base_dir)
itemCounter=0
# Assuming `train_X_sampled` and `train_y_sampled` are your dataset variables and `model` is a trained model.
for i in range(len(train10_X_sampled)):  
    original_vector = train10_X_sampled[i]  # e.g., shape (784,) for MNIST
    train_y_sample = train10_y_sampled[i]
    
    train_X_sample = original_vector.reshape(1, -1)
    
    cuckoo_search = CuckooSearch(n_nests=58, n_iterations=100, dim=original_vector.shape[0],
                                 lower_bound=0, upper_bound=0.5, model=model, X=train_X_sample, y=[train_y_sample])
    cuckoo_search.optimize()
    best_solution = cuckoo_search.get_best_solution()
    
    # Generate the adversarial example.
    X_adv = original_vector + best_solution
    X_adv = np.clip(X_adv, 0, 1)
    
    X_adv_batch = X_adv.reshape(1, -1)
    y_pred_adv = np.argmax(model.predict(X_adv_batch), axis=1)
    
    if y_pred_adv[0] != train_y_sample:
  
        perturbation_magnitude = np.linalg.norm(best_solution)
        
        adversarial_image_filename = f"{i}adv_orig{train_y_sample}_adv{int(y_pred_adv[0])}_mag{perturbation_magnitude}_{i}.png"
        adversarial_image_path = os.path.join(adversarial_dir, adversarial_image_filename)
        
        save_image(X_adv, adversarial_image_path, img_shape=img_shape)
        itemCounter+=1
        
        results.append({
            'Image_Index': i,
            'Original':train_y_sample,
            'Adversarial_Digit': int(y_pred_adv[0]),
            'Perturbation_Magnitude': perturbation_magnitude,
            'Adversarial_Image_Path': adversarial_image_path
        })
        print(f"Image {i} PERTURBED. itemCounter: {itemCounter } perturbation_magnitude: {perturbation_magnitude}")
    else:
        print(f"Image {i} did not result in adversarial misclassification---------------------------------.")

# Save results to an Excel file
df = pd.DataFrame(results)
df.to_excel(r"CSAzeroCau_images/CSA_Zero_cauchy.xlsx")

# Display the results
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Adversarial Samples Results", dataframe=df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
Image 0 PERTURBED. itemCounter: 1 perturbation_magnitude: 8.272412251714087
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image 1 PERTURBED. itemCounter: 2 perturbation_magnitude: 8.284202396487851
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Image 2 PERTURBED. itemCounter: 3 perturbation_magnitude: 8.474188985225247
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image 3 PERTURBED. itemCounter: 4 perturbation_magnitude: 7.882923300715495
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Image 4 PERTURBED. itemCounter: 5 perturbation_magnitude: 8.103013468404152
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Image 5 PERTURBED. itemCounter: 6 perturbation_magnitude: 8.128529108341585
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Image 6 PERTURBED. itemCounter: 7 perturbation_magnitude: 8.12781742960792
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Image 7 PERTURBED. itemCounter: 8 perturbation_magnitude: 8.160704478955653
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Image 8 PERTURBED. itemCounter: 9 perturbation_mag

### Start testing on 500 samples

### On seen data

In [12]:

img_shape = (28, 28) # For MNIST images we use 28x28 grayscale.

# --- 1. Define the Adversarial Loss Function ---
def adversarial_loss(model, x, y):
    x_adv = x
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure values are in [0,1]
    
    # Get predictions and compute loss
    y_pred = model(x_adv, training=False)
    y_true = tf.convert_to_tensor(y, dtype=tf.int64)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss

# --- 2. Cuckoo Search Algorithm for Finding Adversarial Perturbations ---
class CuckooSearch:
    def __init__(self, n_nests, n_iterations, dim, lower_bound, upper_bound, model, X, y, p_a=0.1):
        self.n_nests = n_nests
        self.n_iterations = n_iterations
        self.dim = dim
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.model = model
        self.X = X
        self.y = y
        self.p_a = p_a  # Discovery probability
        
        # Initialize nests (solutions) with no perturbation (Zero)
        self.nests = np.zeros((self.n_nests, self.dim))
        self.fitness = np.array([self.fitness_function(self.nests[i], X, y)
                                 for i in range(self.n_nests)])
        
        self.best_nest = self.nests[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)
    
    def fitness_function(self, perturbation, X, y):
        perturbed_X = X + perturbation
        perturbed_X = np.clip(perturbed_X, self.lower_bound, self.upper_bound)
        loss = adversarial_loss(self.model, perturbed_X, y)  
        perturbation_magnitude = np.linalg.norm(perturbation)
        
        return - loss * perturbation_magnitude
       
    
    def generate_new_solution(self, current_solution, step_size=0.001):
        step = np.random.standard_cauchy(self.dim) * step_size
        new_solution = current_solution + step
        new_solution = np.clip(new_solution, self.lower_bound, self.upper_bound)
        return new_solution
    
    def optimize(self):
        for iteration in range(self.n_iterations):
            # Perturb each nest
            for i in range(self.n_nests):
                new_solution = self.generate_new_solution(self.nests[i], step_size=0.001)
                new_fitness = self.fitness_function(new_solution, self.X, self.y)
                if new_fitness < self.fitness[i]:
                    self.nests[i] = new_solution
                    self.fitness[i] = new_fitness
                    if new_fitness < self.best_fitness:
                        self.best_fitness = new_fitness
                        self.best_nest = new_solution
            
            # Randomly replace some nests with new random solutions
            for i in range(self.n_nests):
                if np.random.rand() < self.p_a:
                    worst_indices = np.argsort(self.fitness)[-int(self.n_nests * self.p_a):]
                    # Replace only the worst nests
                    for idx in worst_indices:
                        self.nests[idx] = np.random.uniform(self.lower_bound, self.upper_bound, self.dim)
                        self.fitness[idx] = self.fitness_function(self.nests[idx], self.X, self.y)
                        
            #print(f"Iteration {iteration+1}/{self.n_iterations}, Best Fitness: {self.best_fitness}")
        
    
    def get_best_solution(self):
        return self.best_nest

# --- 3. Utility Functions for Saving Images and Perturbations ---
def create_directories(base_dir):
    adversarial_dir = os.path.join(base_dir, "Adversarial")
    os.makedirs(adversarial_dir, exist_ok=True)
    
    return adversarial_dir

def save_image(image, file_path, img_shape=None):
    if img_shape is not None and image.size == np.prod(img_shape):
        image = image.reshape(img_shape)
    # Scale [0,1] to [0,255] and convert to uint8
    image_to_save = (image * 255).astype(np.uint8)
    if image_to_save.ndim == 2:
        cv2.imwrite(file_path, image_to_save)
    elif image_to_save.ndim == 3:
        cv2.imwrite(file_path, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


# --- 4. Main Process: Generate, Save, and Record Adversarial Examples ---
results = []
base_dir = "Seen_adversarial_images"
adversarial_dir = create_directories(base_dir)
itemCounter=0
for i in range(len(train_X_sampled)):  
    original_vector = train_X_sampled[i]  # e.g., shape (784,) for MNIST
    train_y_sample = train_y_sampled[i]
    
    train_X_sample = original_vector.reshape(1, -1)
    
    cuckoo_search = CuckooSearch(n_nests=58, n_iterations=100, dim=original_vector.shape[0],
                                 lower_bound=0, upper_bound=0.5, model=model, X=train_X_sample, y=[train_y_sample])
    cuckoo_search.optimize()
    best_solution = cuckoo_search.get_best_solution()
    
    # Generate the adversarial example.
    X_adv = original_vector + best_solution
    X_adv = np.clip(X_adv, 0, 1)
    
    X_adv_batch = X_adv.reshape(1, -1)
    y_pred_adv = np.argmax(model.predict(X_adv_batch), axis=1)
    
    if y_pred_adv[0] != train_y_sample:
  
        perturbation_magnitude = np.linalg.norm(best_solution)
        
        adversarial_image_filename = f"{i}adv_orig{train_y_sample}_adv{int(y_pred_adv[0])}_mag{perturbation_magnitude}_{i}.png"
        adversarial_image_path = os.path.join(adversarial_dir, adversarial_image_filename)
        
        save_image(X_adv, adversarial_image_path, img_shape=img_shape)
        itemCounter+=1
        
        results.append({
            'Image_Index': i,
            'Original':train_y_sample,
            'Adversarial_Digit': int(y_pred_adv[0]),
            'Perturbation_Magnitude': perturbation_magnitude,
            'Adversarial_Image_Path': adversarial_image_path
        })
        print(f"Image {i} PERTURBED. itemCounter: {itemCounter} perturbation_magnitude: {perturbation_magnitude}")
    else:
        print(f"Image {i} did not result in adversarial misclassification.")

# Save results to an Excel file
df = pd.DataFrame(results)
df.to_excel(r"Seen_adversarial_images/Seen_adversarial_images_stats.xlsx")

# Display the results
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Adversarial Samples Results", dataframe=df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image 0 PERTURBED. itemCounter: 1 perturbation_magnitude: 8.352440695060052
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Image 1 PERTURBED. itemCounter: 2 perturbation_magnitude: 8.194073640802712
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Image 2 PERTURBED. itemCounter: 3 perturbation_magnitude: 8.366809212916822
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Image 3 PERTURBED. itemCounter: 4 perturbation_magnitude: 8.470941248128456
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Image 4 PERTURBED. itemCounter: 5 perturbation_magnitude: 8.333823641082237
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Image 5 PERTURBED. itemCounter: 6 perturbation_magnitude: 8.310516626429855
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 6 PERTURBED. itemCounter: 7 perturbation_magnitude: 8.241115224396257
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Image 7 PERTURBED. itemCounter: 8 perturbation_magnitude: 8.225910585191881
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Image 8 PERTURBED. itemCounter: 9 perturbation_mag

### On unseen data

In [14]:

img_shape = (28, 28) # For MNIST images we use 28x28 grayscale.

# --- 1. Define the Adversarial Loss Function ---
def adversarial_loss(model, x, y):
    x_adv = x
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure values are in [0,1]
    
    # Get predictions and compute loss
    y_pred = model(x_adv, training=False)
    y_true = tf.convert_to_tensor(y, dtype=tf.int64)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss

# --- 2. Cuckoo Search Algorithm for Finding Adversarial Perturbations ---
class CuckooSearch:
    def __init__(self, n_nests, n_iterations, dim, lower_bound, upper_bound, model, X, y, p_a=0.1):
        self.n_nests = n_nests
        self.n_iterations = n_iterations
        self.dim = dim
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.model = model
        self.X = X
        self.y = y
        self.p_a = p_a  # Discovery probability
        
        # Initialize nests (solutions) with no perturbation (Zero)
        self.nests = np.zeros((self.n_nests, self.dim))
        self.fitness = np.array([self.fitness_function(self.nests[i], X, y)
                                 for i in range(self.n_nests)])
        
        self.best_nest = self.nests[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)
    
    def fitness_function(self, perturbation, X, y):
        perturbed_X = X + perturbation
        perturbed_X = np.clip(perturbed_X, self.lower_bound, self.upper_bound)
        loss = adversarial_loss(self.model, perturbed_X, y)  
        perturbation_magnitude = np.linalg.norm(perturbation)
        return - loss * perturbation_magnitude 
       
    
    def generate_new_solution(self, current_solution, step_size=0.001):
        step = np.random.standard_cauchy(self.dim) * step_size
        new_solution = current_solution + step
        new_solution = np.clip(new_solution, self.lower_bound, self.upper_bound)
        return new_solution
    
    def optimize(self):
        for iteration in range(self.n_iterations):
            # Perturb each nest
            for i in range(self.n_nests):
                new_solution = self.generate_new_solution(self.nests[i], step_size=0.001)
                new_fitness = self.fitness_function(new_solution, self.X, self.y)
                if new_fitness < self.fitness[i]:
                    self.nests[i] = new_solution
                    self.fitness[i] = new_fitness
                    if new_fitness < self.best_fitness:
                        self.best_fitness = new_fitness
                        self.best_nest = new_solution
            
            # Randomly replace some nests with new random solutions
            for i in range(self.n_nests):
                if np.random.rand() < self.p_a:
                    worst_indices = np.argsort(self.fitness)[-int(self.n_nests * self.p_a):]
                    # Replace only the worst nests
                    for idx in worst_indices:
                        self.nests[idx] = np.random.uniform(self.lower_bound, self.upper_bound, self.dim)
                        self.fitness[idx] = self.fitness_function(self.nests[idx], self.X, self.y)
                        
            #print(f"Iteration {iteration+1}/{self.n_iterations}, Best Fitness: {self.best_fitness}")
        
    
    def get_best_solution(self):
        return self.best_nest

# --- 3. Utility Functions for Saving Images and Perturbations ---
def create_directories(base_dir):
    adversarial_dir = os.path.join(base_dir, "Adversarial")
    os.makedirs(adversarial_dir, exist_ok=True)
    
    return adversarial_dir

def save_image(image, file_path, img_shape=None):
    if img_shape is not None and image.size == np.prod(img_shape):
        image = image.reshape(img_shape)
    # Scale [0,1] to [0,255] and convert to uint8
    image_to_save = (image * 255).astype(np.uint8)
    if image_to_save.ndim == 2:
        cv2.imwrite(file_path, image_to_save)
    elif image_to_save.ndim == 3:
        cv2.imwrite(file_path, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


# --- 4. Main Process: Generate, Save, and Record Adversarial Examples ---
results = []
base_dir = "Unseen_adversarial_images"
adversarial_dir = create_directories(base_dir)
itemCounter=0
# Assuming `X_sampled` and `y_sampled` are your dataset variables and `model` is a trained model.
for i in range(len(X_sampled)):  
    original_vector = X_sampled[i]  # e.g., shape (784,) for MNIST
    y_sample = y_sampled[i]
    
    X_sample = original_vector.reshape(1, -1)
    
    cuckoo_search = CuckooSearch(n_nests=58, n_iterations=100, dim=original_vector.shape[0],
                                 lower_bound=0, upper_bound=0.5, model=model, X=X_sample, y=[y_sample])
    cuckoo_search.optimize()
    best_solution = cuckoo_search.get_best_solution()
    
    # Generate the adversarial example.
    X_adv = original_vector + best_solution
    X_adv = np.clip(X_adv, 0, 1)
    
    X_adv_batch = X_adv.reshape(1, -1)
    y_pred_adv = np.argmax(model.predict(X_adv_batch), axis=1)
    
    if y_pred_adv[0] != y_sample:
  
        perturbation_magnitude = np.linalg.norm(best_solution)
        
        adversarial_image_filename = f"{i}adv_orig{y_sample}_adv{int(y_pred_adv[0])}_mag{perturbation_magnitude}_{i}.png"
        adversarial_image_path = os.path.join(adversarial_dir, adversarial_image_filename)
        
        save_image(X_adv, adversarial_image_path, img_shape=img_shape)
        itemCounter+=1
        
        results.append({
            'Image_Index': i,
            'Original':y_sample,
            'Adversarial_Digit': int(y_pred_adv[0]),
            'Perturbation_Magnitude': perturbation_magnitude,
            'Adversarial_Image_Path': adversarial_image_path
        })
        print(f"Image {i} PERTURBED. itemCounter: {itemCounter} perturbation_magnitude: {perturbation_magnitude}")
    else:
        print(f"Image {i} did not result in adversarial misclassification.")

# Save results to an Excel file
df = pd.DataFrame(results)
df.to_excel(r"Unseen_adversarial_images/Unseen_adversarial_images_stats.xlsx")

# Display the results
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Adversarial Samples Results", dataframe=df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 0 PERTURBED. itemCounter: 1 perturbation_magnitude: 8.266157530566609
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Image 1 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Image 2 PERTURBED. itemCounter: 2 perturbation_magnitude: 8.472701016273518
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 3 PERTURBED. itemCounter: 3 perturbation_magnitude: 8.142942270585289
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 4 PERTURBED. itemCounter: 4 perturbation_magnitude: 8.008409761973887
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 5 PERTURBED. itemCounter: 5 perturbation_magnitude: 8.314616826343372
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Image 6 PERTURBED. itemCounter: 6 perturbation_magnitude: 8.399922479261608
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Image 7 PERTURBED. itemCounter: 7 perturbation_magnitude: 8.119467715090684
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 8 PERTURBED. itemCounter: 8 perturbation_magnitude: 8.314233299

### UnSeen data with low query 10 nests and 10 iterations

In [14]:

img_shape = (28, 28) # For MNIST images we use 28x28 grayscale.

# --- 1. Define the Adversarial Loss Function ---
def adversarial_loss(model, x, y):
    x_adv = x
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure values are in [0,1]
    
    # Get predictions and compute loss
    y_pred = model(x_adv, training=False)
    y_true = tf.convert_to_tensor(y, dtype=tf.int64)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss

# --- 2. Cuckoo Search Algorithm for Finding Adversarial Perturbations ---
class CuckooSearch:
    def __init__(self, n_nests, n_iterations, dim, lower_bound, upper_bound, model, X, y, p_a=0.1):
        self.n_nests = n_nests
        self.n_iterations = n_iterations
        self.dim = dim
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.model = model
        self.X = X
        self.y = y
        self.p_a = p_a  # Discovery probability
        
        # Initialize nests (solutions) with no perturbation (Zero)
        self.nests = np.zeros((self.n_nests, self.dim))
        self.fitness = np.array([self.fitness_function(self.nests[i], X, y)
                                 for i in range(self.n_nests)])
        
        self.best_nest = self.nests[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)
    
    def fitness_function(self, perturbation, X, y):
        perturbed_X = X + perturbation
        perturbed_X = np.clip(perturbed_X, self.lower_bound, self.upper_bound)
        loss = adversarial_loss(self.model, perturbed_X, y)  
        perturbation_magnitude = np.linalg.norm(perturbation)
        return - loss * perturbation_magnitude 
       
    
    def generate_new_solution(self, current_solution, step_size=0.001):
        step = np.random.standard_cauchy(self.dim) * step_size
        new_solution = current_solution + step
        new_solution = np.clip(new_solution, self.lower_bound, self.upper_bound)
        return new_solution
    
    def optimize(self):
        for iteration in range(self.n_iterations):
            # Perturb each nest
            for i in range(self.n_nests):
                new_solution = self.generate_new_solution(self.nests[i], step_size=0.001)
                new_fitness = self.fitness_function(new_solution, self.X, self.y)
                if new_fitness < self.fitness[i]:
                    self.nests[i] = new_solution
                    self.fitness[i] = new_fitness
                    if new_fitness < self.best_fitness:
                        self.best_fitness = new_fitness
                        self.best_nest = new_solution
            
            # Randomly replace some nests with new random solutions
            for i in range(self.n_nests):
                if np.random.rand() < self.p_a:
                    worst_indices = np.argsort(self.fitness)[-int(self.n_nests * self.p_a):]
                    # Replace only the worst nests
                    for idx in worst_indices:
                        self.nests[idx] = np.random.uniform(self.lower_bound, self.upper_bound, self.dim)
                        self.fitness[idx] = self.fitness_function(self.nests[idx], self.X, self.y)
                        
            #print(f"Iteration {iteration+1}/{self.n_iterations}, Best Fitness: {self.best_fitness}")
        
    
    def get_best_solution(self):
        return self.best_nest

# --- 3. Utility Functions for Saving Images and Perturbations ---
def create_directories(base_dir):
    adversarial_dir = os.path.join(base_dir, "Adversarial")
    os.makedirs(adversarial_dir, exist_ok=True)
    
    return adversarial_dir

def save_image(image, file_path, img_shape=None):
    if img_shape is not None and image.size == np.prod(img_shape):
        image = image.reshape(img_shape)
    # Scale [0,1] to [0,255] and convert to uint8
    image_to_save = (image * 255).astype(np.uint8)
    if image_to_save.ndim == 2:
        cv2.imwrite(file_path, image_to_save)
    elif image_to_save.ndim == 3:
        cv2.imwrite(file_path, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


# --- 4. Main Process: Generate, Save, and Record Adversarial Examples ---
results = []
base_dir = "LowQuery_Unseen_adversarial_images"
adversarial_dir = create_directories(base_dir)
itemCounter=0
# Assuming `X_sampled` and `y_sampled` are your dataset variables and `model` is a trained model.
for i in range(len(X_sampled)):  
    original_vector = X_sampled[i]  # e.g., shape (784,) for MNIST
    y_sample = y_sampled[i]
    
    X_sample = original_vector.reshape(1, -1)
    
    cuckoo_search = CuckooSearch(n_nests=10, n_iterations=10, dim=original_vector.shape[0],
                                 lower_bound=0, upper_bound=0.5, model=model, X=X_sample, y=[y_sample])
    cuckoo_search.optimize()
    best_solution = cuckoo_search.get_best_solution()
    
    # Generate the adversarial example.
    X_adv = original_vector + best_solution
    X_adv = np.clip(X_adv, 0, 1)
    
    X_adv_batch = X_adv.reshape(1, -1)
    y_pred_adv = np.argmax(model.predict(X_adv_batch), axis=1)
    
    if y_pred_adv[0] != y_sample:
  
        perturbation_magnitude = np.linalg.norm(best_solution)
        
        adversarial_image_filename = f"{i}adv_orig{y_sample}_adv{int(y_pred_adv[0])}_mag{perturbation_magnitude}_{i}.png"
        adversarial_image_path = os.path.join(adversarial_dir, adversarial_image_filename)
        
        save_image(X_adv, adversarial_image_path, img_shape=img_shape)
        itemCounter+=1
        
        results.append({
            'Image_Index': i,
            'Original':y_sample,
            'Adversarial_Digit': int(y_pred_adv[0]),
            'Perturbation_Magnitude': perturbation_magnitude,
            'Adversarial_Image_Path': adversarial_image_path
        })
        print(f"Image {i} PERTURBED. itemCounter: {itemCounter} perturbation_magnitude: {perturbation_magnitude}")
    else:
        print(f"Image {i} did not result in adversarial misclassification.")

# Save results to an Excel file
df = pd.DataFrame(results)
df.to_excel(r"LowQuery_Unseen_adversarial_images/LowQuery_Unseen_adversarial_images_stats.xlsx")

# Display the results
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Adversarial Samples Results", dataframe=df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Image 0 PERTURBED. itemCounter: 1 perturbation_magnitude: 8.199596061753445
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Image 1 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image 2 PERTURBED. itemCounter: 2 perturbation_magnitude: 8.164610545819048
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image 3 PERTURBED. itemCounter: 3 perturbation_magnitude: 8.271076352045503
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Image 4 PERTURBED. itemCounter: 4 perturbation_magnitude: 8.156161279865286
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Image 5 PERTURBED. itemCounter: 5 perturbation_magnitude: 8.339380398223351
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Image 6 PERTURBED. itemCounter: 6 perturbation_magnitude: 7.915576179538585
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Image 7 PERTURBED. itemCounter: 7 perturbation_magnitude: 8.262697364825563
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image 8 PERTURBED. itemCounter: 8 perturbation_magnitude: 8.150118770

### UnSeen data with low magnitude and low query 10 nests and 10 iterations

In [16]:

img_shape = (28, 28) # For MNIST images we use 28x28 grayscale.

# --- 1. Define the Adversarial Loss Function ---
def adversarial_loss(model, x, y):
    x_adv = x
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure values are in [0,1]
    
    # Get predictions and compute loss
    y_pred = model(x_adv, training=False)
    y_true = tf.convert_to_tensor(y, dtype=tf.int64)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss

# --- 2. Cuckoo Search Algorithm for Finding Adversarial Perturbations ---
class CuckooSearch:
    def __init__(self, n_nests, n_iterations, dim, lower_bound, upper_bound, model, X, y, p_a=0.1):
        self.n_nests = n_nests
        self.n_iterations = n_iterations
        self.dim = dim
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.model = model
        self.X = X
        self.y = y
        self.p_a = p_a  # Discovery probability
        
        # Initialize nests (solutions) with no perturbation (Zero)
        self.nests = np.zeros((self.n_nests, self.dim))
        self.fitness = np.array([self.fitness_function(self.nests[i], X, y)
                                 for i in range(self.n_nests)])
        
        self.best_nest = self.nests[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)
    
    def fitness_function(self, perturbation, X, y):
        perturbed_X = X + perturbation
        perturbed_X = np.clip(perturbed_X, self.lower_bound, self.upper_bound)
        loss = adversarial_loss(self.model, perturbed_X, y)  
        perturbation_magnitude = np.linalg.norm(perturbation)
        return - loss * perturbation_magnitude 
       
    
    def generate_new_solution(self, current_solution, step_size=0.001):
        step = np.random.standard_cauchy(self.dim) * step_size
        new_solution = current_solution + step
        new_solution = np.clip(new_solution, self.lower_bound, self.upper_bound)
        return new_solution
    
    def optimize(self):
        for iteration in range(self.n_iterations):
            # Perturb each nest
            for i in range(self.n_nests):
                new_solution = self.generate_new_solution(self.nests[i], step_size=0.001)
                new_fitness = self.fitness_function(new_solution, self.X, self.y)
                if new_fitness < self.fitness[i]:
                    self.nests[i] = new_solution
                    self.fitness[i] = new_fitness
                    if new_fitness < self.best_fitness:
                        self.best_fitness = new_fitness
                        self.best_nest = new_solution
            
            # Randomly replace some nests with new random solutions
            for i in range(self.n_nests):
                if np.random.rand() < self.p_a:
                    worst_indices = np.argsort(self.fitness)[-int(self.n_nests * self.p_a):]
                    # Replace only the worst nests
                    for idx in worst_indices:
                        self.nests[idx] = np.random.uniform(self.lower_bound, self.upper_bound, self.dim)
                        self.fitness[idx] = self.fitness_function(self.nests[idx], self.X, self.y)
                        
            #print(f"Iteration {iteration+1}/{self.n_iterations}, Best Fitness: {self.best_fitness}")
        
    
    def get_best_solution(self):
        return self.best_nest

# --- 3. Utility Functions for Saving Images and Perturbations ---
def create_directories(base_dir):
    adversarial_dir = os.path.join(base_dir, "Adversarial")
    os.makedirs(adversarial_dir, exist_ok=True)
    
    return adversarial_dir

def save_image(image, file_path, img_shape=None):
    if img_shape is not None and image.size == np.prod(img_shape):
        image = image.reshape(img_shape)
    # Scale [0,1] to [0,255] and convert to uint8
    image_to_save = (image * 255).astype(np.uint8)
    if image_to_save.ndim == 2:
        cv2.imwrite(file_path, image_to_save)
    elif image_to_save.ndim == 3:
        cv2.imwrite(file_path, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


# --- 4. Main Process: Generate, Save, and Record Adversarial Examples ---
results = []
base_dir = "LowMag_LowQuery_Unseen_adversarial_images"
adversarial_dir = create_directories(base_dir)
itemCounter=0
# Assuming `X_sampled` and `y_sampled` are your dataset variables and `model` is a trained model.
for i in range(len(X_sampled)):  
    original_vector = X_sampled[i]  # e.g., shape (784,) for MNIST
    y_sample = y_sampled[i]
    
    X_sample = original_vector.reshape(1, -1)
    
    cuckoo_search = CuckooSearch(n_nests=10, n_iterations=10, dim=original_vector.shape[0],
                                 lower_bound=0, upper_bound=0.01, model=model, X=X_sample, y=[y_sample])
    cuckoo_search.optimize()
    best_solution = cuckoo_search.get_best_solution()
    
    # Generate the adversarial example.
    X_adv = original_vector + best_solution
    X_adv = np.clip(X_adv, 0, 1)
    
    X_adv_batch = X_adv.reshape(1, -1)
    y_pred_adv = np.argmax(model.predict(X_adv_batch), axis=1)
    
    if y_pred_adv[0] != y_sample:
  
        perturbation_magnitude = np.linalg.norm(best_solution)
        
        adversarial_image_filename = f"{i}adv_orig{y_sample}_adv{int(y_pred_adv[0])}_mag{perturbation_magnitude}_{i}.png"
        adversarial_image_path = os.path.join(adversarial_dir, adversarial_image_filename)
        
        save_image(X_adv, adversarial_image_path, img_shape=img_shape)
        itemCounter+=1
        
        results.append({
            'Image_Index': i,
            'Original':y_sample,
            'Adversarial_Digit': int(y_pred_adv[0]),
            'Perturbation_Magnitude': perturbation_magnitude,
            'Adversarial_Image_Path': adversarial_image_path
        })
        print(f"Image {i} PERTURBED. itemCounter: {itemCounter} perturbation_magnitude: {perturbation_magnitude}")
    else:
        print(f"Image {i} did not result in adversarial misclassification.")

# Save results to an Excel file
df = pd.DataFrame(results)
df.to_excel(r"LowMag_LowQuery_Unseen_adversarial_images/LowMag_LowQuery_Unseen_adversarial_images_stats.xlsx")

# Display the results
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Adversarial Samples Results", dataframe=df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Image 0 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 1 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image 2 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Image 3 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Image 4 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Image 5 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Image 6 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Image 7 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Image 8 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Image 9 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Image 10 did

### UnSeen data with extreme low magnitude and low query 10 nests and 10 iterations

In [8]:

img_shape = (28, 28) # For MNIST images we use 28x28 grayscale.

# --- 1. Define the Adversarial Loss Function ---
def adversarial_loss(model, x, y):
    x_adv = x
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure values are in [0,1]
    
    # Get predictions and compute loss
    y_pred = model(x_adv, training=False)
    y_true = tf.convert_to_tensor(y, dtype=tf.int64)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss

# --- 2. Cuckoo Search Algorithm for Finding Adversarial Perturbations ---
class CuckooSearch:
    def __init__(self, n_nests, n_iterations, dim, lower_bound, upper_bound, model, X, y, p_a=0.1):
        self.n_nests = n_nests
        self.n_iterations = n_iterations
        self.dim = dim
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.model = model
        self.X = X
        self.y = y
        self.p_a = p_a  # Discovery probability
        
        # Initialize nests (solutions) with no perturbation (Zero)
        self.nests = np.zeros((self.n_nests, self.dim))
        self.fitness = np.array([self.fitness_function(self.nests[i], X, y)
                                 for i in range(self.n_nests)])
        
        self.best_nest = self.nests[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)
    
    def fitness_function(self, perturbation, X, y):
        perturbed_X = X + perturbation
        perturbed_X = np.clip(perturbed_X, self.lower_bound, self.upper_bound)
        loss = adversarial_loss(self.model, perturbed_X, y)  
        perturbation_magnitude = np.linalg.norm(perturbation)
        return - loss * perturbation_magnitude 
       
    
    def generate_new_solution(self, current_solution, step_size=0.00001):
        step = np.random.standard_cauchy(self.dim) * step_size
        new_solution = current_solution + step
        new_solution = np.clip(new_solution, self.lower_bound, self.upper_bound)
        return new_solution
    
    def optimize(self):
        for iteration in range(self.n_iterations):
            # Perturb each nest
            for i in range(self.n_nests):
                new_solution = self.generate_new_solution(self.nests[i], step_size=0.00001)
                new_fitness = self.fitness_function(new_solution, self.X, self.y)
                if new_fitness < self.fitness[i]:
                    self.nests[i] = new_solution
                    self.fitness[i] = new_fitness
                    if new_fitness < self.best_fitness:
                        self.best_fitness = new_fitness
                        self.best_nest = new_solution
            
            # Randomly replace some nests with new random solutions
            for i in range(self.n_nests):
                if np.random.rand() < self.p_a:
                    worst_indices = np.argsort(self.fitness)[-int(self.n_nests * self.p_a):]
                    # Replace only the worst nests
                    for idx in worst_indices:
                        self.nests[idx] = np.random.uniform(self.lower_bound, self.upper_bound, self.dim)
                        self.fitness[idx] = self.fitness_function(self.nests[idx], self.X, self.y)
                        
            #print(f"Iteration {iteration+1}/{self.n_iterations}, Best Fitness: {self.best_fitness}")
        
    
    def get_best_solution(self):
        return self.best_nest

# --- 3. Utility Functions for Saving Images and Perturbations ---
def create_directories(base_dir):
    adversarial_dir = os.path.join(base_dir, "Adversarial")
    os.makedirs(adversarial_dir, exist_ok=True)
    
    return adversarial_dir

def save_image(image, file_path, img_shape=None):
    if img_shape is not None and image.size == np.prod(img_shape):
        image = image.reshape(img_shape)
    # Scale [0,1] to [0,255] and convert to uint8
    image_to_save = (image * 255).astype(np.uint8)
    if image_to_save.ndim == 2:
        cv2.imwrite(file_path, image_to_save)
    elif image_to_save.ndim == 3:
        cv2.imwrite(file_path, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


# --- 4. Main Process: Generate, Save, and Record Adversarial Examples ---
results = []
base_dir = "ExLowMag_LowQuery_Unseen_adversarial_images"
adversarial_dir = create_directories(base_dir)
itemCounter=0
# Assuming `X_sampled` and `y_sampled` are your dataset variables and `model` is a trained model.
for i in range(len(X_sampled)):  
    original_vector = X_sampled[i]  # e.g., shape (784,) for MNIST
    y_sample = y_sampled[i]
    
    X_sample = original_vector.reshape(1, -1)
    
    cuckoo_search = CuckooSearch(n_nests=10, n_iterations=10, dim=original_vector.shape[0],
                                 lower_bound=0, upper_bound=0.00001, model=model, X=X_sample, y=[y_sample])
    cuckoo_search.optimize()
    best_solution = cuckoo_search.get_best_solution()
    
    # Generate the adversarial example.
    X_adv = original_vector + best_solution
    X_adv = np.clip(X_adv, 0, 1)
    
    X_adv_batch = X_adv.reshape(1, -1)
    y_pred_adv = np.argmax(model.predict(X_adv_batch), axis=1)
    
    if y_pred_adv[0] != y_sample:
  
        perturbation_magnitude = np.linalg.norm(best_solution)
        
        adversarial_image_filename = f"{i}adv_orig{y_sample}_adv{int(y_pred_adv[0])}_mag{perturbation_magnitude}_{i}.png"
        adversarial_image_path = os.path.join(adversarial_dir, adversarial_image_filename)
        
        save_image(X_adv, adversarial_image_path, img_shape=img_shape)
        itemCounter+=1
        
        results.append({
            'Image_Index': i,
            'Original':y_sample,
            'Adversarial_Digit': int(y_pred_adv[0]),
            'Perturbation_Magnitude': perturbation_magnitude,
            'Adversarial_Image_Path': adversarial_image_path
        })
        print(f"Image {i} PERTURBED. itemCounter: {itemCounter} perturbation_magnitude: {perturbation_magnitude}")
    else:
        print(f"Image {i} did not result in adversarial misclassification.")

# Save results to an Excel file
df = pd.DataFrame(results)
df.to_excel(r"ExLowMag_LowQuery_Unseen_adversarial_images/ExLowMag_LowQuery_Unseen_adversarial_images_stats.xlsx")

# Display the results
import ace_tools_open as tools
tools.display_dataframe_to_user(name="Adversarial Samples Results", dataframe=df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Image 0 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Image 1 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Image 2 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image 3 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 4 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Image 5 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Image 6 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Image 7 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Image 8 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Image 9 did not result in adversarial misclassification.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Image 10 did

### Changing to CNN

In [48]:
import tensorflow as tf
import numpy as np
import cv2
import os
import joblib
import pandas as pd
import ace_tools_open as tools

# Loading the pre-trained CNN model 
model = joblib.load(r"CNN_2L.pkl")

img_shape = (28, 28, 1)  # For MNIST images we use 28x28 grayscale with a single channel.

# --- 1. Define the Adversarial Loss Function ---
def adversarial_loss(model, x, y):
    x_adv = x
    x_adv = tf.clip_by_value(x_adv, 0, 1)  # Ensure values are in [0,1]
    
    # Get predictions and compute loss
    y_pred = model(x_adv, training=False)
    y_true = tf.convert_to_tensor(y, dtype=tf.int64)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    return loss

# --- 2. Cuckoo Search Algorithm for Finding Adversarial Perturbations ---
class CuckooSearch:
    def __init__(self, n_nests, n_iterations, dim, lower_bound, upper_bound, model, X, y, p_a=0.1):
        self.n_nests = n_nests
        self.n_iterations = n_iterations
        self.dim = dim
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.model = model
        self.X = X
        self.y = y
        self.p_a = p_a  # Discovery probability
        
        # Initialize nests (solutions) with no perturbation (Zero)
        self.nests = np.zeros((self.n_nests, self.dim))
        self.fitness = np.array([self.fitness_function(self.nests[i], X, y)
                                 for i in range(self.n_nests)])
        
        self.best_nest = self.nests[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)
    
    def fitness_function(self, perturbation, X, y):
        # Reshape the perturbation to match image dimensions
        perturbed_X = X + perturbation.reshape(*X.shape)  # Add perturbation to image
        perturbed_X = np.clip(perturbed_X, self.lower_bound, self.upper_bound)
        loss = adversarial_loss(self.model, perturbed_X, y)  
        perturbation_magnitude = np.linalg.norm(perturbation)
        return - loss * perturbation_magnitude 
       
    
    def generate_new_solution(self, current_solution, step_size=0.00001):
        step = np.random.standard_cauchy(self.dim) * step_size
        new_solution = current_solution + step
        new_solution = np.clip(new_solution, self.lower_bound, self.upper_bound)
        return new_solution
    
    def optimize(self):
        for iteration in range(self.n_iterations):
            # Perturb each nest
            for i in range(self.n_nests):
                new_solution = self.generate_new_solution(self.nests[i], step_size=0.00001)
                new_fitness = self.fitness_function(new_solution, self.X, self.y)
                if new_fitness < self.fitness[i]:
                    self.nests[i] = new_solution
                    self.fitness[i] = new_fitness
                    if new_fitness < self.best_fitness:
                        self.best_fitness = new_fitness
                        self.best_nest = new_solution
            
            # Randomly replace some nests with new random solutions
            for i in range(self.n_nests):
                if np.random.rand() < self.p_a:
                    worst_indices = np.argsort(self.fitness)[-int(self.n_nests * self.p_a):]
                    # Replace only the worst nests
                    for idx in worst_indices:
                        self.nests[idx] = np.random.uniform(self.lower_bound, self.upper_bound, self.dim)
                        self.fitness[idx] = self.fitness_function(self.nests[idx], self.X, self.y)
                        
            print(f"Iteration {iteration+1}/{self.n_iterations}, Best Fitness: {self.best_fitness}")
        
    
    def get_best_solution(self):
        return self.best_nest

# --- 3. Utility Functions for Saving Images and Perturbations ---
def create_directories(base_dir):
    adversarial_dir = os.path.join(base_dir, "Adversarial")
    os.makedirs(adversarial_dir, exist_ok=True)
    
    return adversarial_dir

def save_image(image, file_path, img_shape=None):
    if img_shape is not None and image.size == np.prod(img_shape):
        image = image.reshape(img_shape)
    # Scale [0,1] to [0,255] and convert to uint8
    image_to_save = (image * 255).astype(np.uint8)
    if image_to_save.ndim == 2:
        cv2.imwrite(file_path, image_to_save)
    elif image_to_save.ndim == 3:
        cv2.imwrite(file_path, cv2.cvtColor(image_to_save, cv2.COLOR_RGB2BGR))


# --- 4. Main Process: Generate, Save, and Record Adversarial Examples ---
results = []
base_dir = "CNN_ExLowMag_LowQuery_Unseen_adversarial_images"
adversarial_dir = create_directories(base_dir)
itemCounter = 0

# Assuming `X_sampled` and `y_sampled` are your dataset variables and `model` is a trained model.
for i in range(len(X_sampled)):  
    original_image = X_sampled[i]  # e.g., shape (28, 28, 1) for MNIST
    y_sample = y_sampled[i]
    
    X_sample = original_image.reshape(1, 28, 28, 1)  # Reshape to (1, 28, 28, 1) for CNN input
    
    cuckoo_search = CuckooSearch(n_nests=10, n_iterations=10, dim=np.prod(original_image.shape),
                                 lower_bound=0, upper_bound=0.00001, model=model, X=X_sample, y=[y_sample])
    cuckoo_search.optimize()
    best_solution = cuckoo_search.get_best_solution()
    
    # Generate the adversarial example.
    X_adv = original_image + best_solution.reshape(*original_image.shape)  # Ensure correct reshaping
    X_adv = np.clip(X_adv, 0, 1)
    
    X_adv_batch = X_adv.reshape(1, 28, 28, 1)
    y_pred_adv = np.argmax(model.predict(X_adv_batch), axis=1)
    
    if y_pred_adv[0] != y_sample:
        perturbation_magnitude = np.linalg.norm(best_solution)
        
        adversarial_image_filename = f"{i}adv_orig{y_sample}_adv{int(y_pred_adv[0])}_mag{perturbation_magnitude}_{i}.png"
        adversarial_image_path = os.path.join(adversarial_dir, adversarial_image_filename)
        
        save_image(X_adv, adversarial_image_path, img_shape=img_shape)
        itemCounter += 1
        
        results.append({
            'Image_Index': i,
            'Original': y_sample,
            'Adversarial_Digit': int(y_pred_adv[0]),
            'Perturbation_Magnitude': perturbation_magnitude,
            'Adversarial_Image_Path': adversarial_image_path
        })
        print(f"Image {i} PERTURBED. itemCounter: {itemCounter} perturbation_magnitude: {perturbation_magnitude}")
    else:
        print(f"Image {i} did not result in adversarial misclassification.")

# Save results to an Excel file
df = pd.DataFrame(results)
df.to_excel(r"CNN_ExLowMag_LowQuery_Unseen_adversarial_images/CNN_ExLowMag_LowQuery_Unseen_adversarial_images_stats.xlsx")

# Display the results
tools.display_dataframe_to_user(name="Adversarial Samples Results", dataframe=df)


Iteration 1/10, Best Fitness: [-0.00041688]
Iteration 2/10, Best Fitness: [-0.00047314]
Iteration 3/10, Best Fitness: [-0.00048442]
Iteration 4/10, Best Fitness: [-0.00049007]
Iteration 5/10, Best Fitness: [-0.00049162]
Iteration 6/10, Best Fitness: [-0.00049162]
Iteration 7/10, Best Fitness: [-0.00049162]
Iteration 8/10, Best Fitness: [-0.00049162]
Iteration 9/10, Best Fitness: [-0.00049801]
Iteration 10/10, Best Fitness: [-0.00049801]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Image 0 did not result in adversarial misclassification.
Iteration 1/10, Best Fitness: [-0.00040774]
Iteration 2/10, Best Fitness: [-0.0004851]
Iteration 3/10, Best Fitness: [-0.00049225]
Iteration 4/10, Best Fitness: [-0.00049225]
Iteration 5/10, Best Fitness: [-0.00049225]
Iteration 6/10, Best Fitness: [-0.00049454]
Iteration 7/10, Best Fitness: [-0.00049454]
Iteration 8/10, Best Fitness: [-0.00049454]
Iteration 9/10, Best Fitness: [-0.00049454]
Iteration 10/10, Best Fitness: [-0.00049454]
1/1 ━━━━━━━━━━━━━━━━━━━━